In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms
from torchvision import datasets
from torchvision import models

import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

from ignite.engine import Engine, Events
from ignite.metrics import Loss, Accuracy

import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint

import shutil
import os

In [2]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.RandomRotation(20),
                                       transforms.RandomHorizontalFlip(0.3),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder('/kaggle/input/gestos/gestos/train', transform=train_transforms)
valid_dataset = datasets.ImageFolder('/kaggle/input/gestos/gestos/valid', transform=valid_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=True)

model = models.resnet18(pretrained=True, progress=True)

In [3]:
#Congelamos todos los parámetros
for param in model.parameters(): 
    param.requires_grad = False

# Recuperamos el número de neuronas de la última capa
neurons = model.fc.in_features 
# La reemplazamos por una nueva capa de salida
model.fc = torch.nn.Linear(neurons, 4) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Esto es lo que hace el engine de entrenamiento
def train_one_step(engine, batch):
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    yhat = model.forward(x)
    loss = criterion(yhat, y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Esto es lo que hace el engine de evaluación
def evaluate_one_step(engine, batch):
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        yhat = model.forward(x)
        return yhat, y

trainer = Engine(train_one_step)
evaluator = Engine(evaluate_one_step)
metrics = {'Loss': Loss(criterion), 'Acc': Accuracy()}
for name, metric in metrics.items():
    metric.attach(evaluator, name)

KeyboardInterrupt: 

In [ ]:
import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint

# Contexto de escritura de datos para tensorboard
with SummaryWriter(log_dir=f'/kaggle/working/tensor') as writer:

    @trainer.on(Events.EPOCH_COMPLETED(every=1)) # Cada 1 epocas
    def log_results(engine):
        # Evaluo el conjunto de entrenamiento
        evaluator.run(train_loader) 
        writer.add_scalar("train/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("train/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        # Evaluo el conjunto de validación
        evaluator.run(valid_loader) 
        writer.add_scalar("valid/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("valid/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
    # Guardo el mejor modelo en validación
    best_model_handler = ModelCheckpoint(dirname='./', require_empty=False, filename_prefix="best", n_saved=5,
                                         score_function=lambda engine: -engine.state.metrics['Loss'],
                                         score_name="val_loss")

    # Lo siguiente se ejecuta cada ves que termine el loop de validación
    evaluator.add_event_handler(Events.COMPLETED, 
                                best_model_handler, {'ResNet18': model})

    trainer.run(train_loader, max_epochs=200)